In [11]:
!pip install webdriver-manager
!pip install pathos

In [12]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager("114.0.5735.16").install()), options=options)
# url 은 삭제하였습니다.
driver.get(url='')

[WDM] - Downloading: 100%|██████████| 7.40M/7.40M [00:00<00:00, 9.40MB/s]


WebDriverException: Message: Service /Users/soyoung/.wdm/drivers/chromedriver/mac_arm64/114.0.5735.16/chromedriver unexpectedly exited. Status code was: -9


In [ ]:
import json
import time
from collections import defaultdict
from tqdm import tqdm
import re

pagination = driver.find_element(By.CLASS_NAME, 'page_arrow.r2')
final_page = int(re.findall(r'\d+', pagination.get_attribute('href')).pop())

# parsing 후 dict 만든 뒤 json file 로 저장
companies = defaultdict(str)
for i in tqdm(range(1, final_page+1)):
    while True:
        try:
            driver.execute_script(f'javascript:PagingHelper.gotoPage({i})')
            time.sleep(0.5)
            table = driver.find_element(By.ID, 'table')
            trs = table.find_elements(By.CLASS_NAME, 'c_tit')
        
            if len(trs) == 0:
                raise
            
            for t in trs:
                company = t.text
                link = t.get_attribute('href')
                vniaSn = link.split('vniaSn=')[1].split('&')[0]
                companies[company] = link
            break
        except:
            pass

with open('company.json', 'w') as f:
    json.dump(companies, f, ensure_ascii = False)


  0%|          | 0/2524 [00:00<?, ?it/s]

100%|██████████| 2524/2524 [35:42<00:00,  1.18it/s]  


In [27]:
from selenium.common.exceptions import NoSuchElementException

companies_js = defaultdict(dict)
for company, link in tqdm(companies.items()):
    try:
        driver.get(url=link)
        time.sleep(0.3)
        ## 일반정보 페이지
        # 업종, 주생산품
        sector = driver.find_element(By.XPATH, '//*[@id="real_contents"]/div/div[1]/div[1]/table/tbody/tr[3]/td[1]').text
        companies_js[company]['sector'] = sector
        product = driver.find_element(By.XPATH, '//*[@id="real_contents"]/div/div[1]/div[1]/table/tbody/tr[3]/td[2]').text
        companies_js[company]['product'] = product

        ## 재무정보 > 대차대조표 페이지 이동
        driver.find_element(By.XPATH, '//*[@id="real_contents"]/div/ul/li[2]/a').click()
        time.sleep(0.3)
        # 마지막 연도 
        latest_year = driver.find_element(By.XPATH, '//*[@id="real_contents"]/div/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[1]/table/thead/tr/th[2]').text
        companies_js[company]['latest_year'] = latest_year

        # 대차대조표 - 부채총계, 현금 및 현금성자산, 자본총계
        liab = []
        cash = []
        equi = []

        for i in range(3):
            liab.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[23]/td[{i+1}]').text)
            cash.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[3]/td[{i+1}]').text)
            equi.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[26]/td[{i+1}]').text)
        companies_js[company]['liab'] = liab
        companies_js[company]['cash'] = cash
        companies_js[company]['equi'] = equi

        ## 재무정보 > 손익계산서 페이지 이동
        driver.find_element(By.XPATH, '//*[@id="real_contents"]/div/div[1]/div[2]/ul/li[2]/a').click()
        time.sleep(0.3)
        # 손익계산서 - 매출액, 영업손익, 감가상각비, 당기순손익
        reve = []
        oper = []
        depr = []
        neti = []
        for i in range(3):
            reve.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[{i+1}]').text)
            oper.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[2]/div[2]/table/tbody/tr[51]/td[{i+1}]').text)
            depr.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[2]/div[2]/table/tbody/tr[33]/td[{i+1}]').text)
            neti.append(driver.find_element(By.XPATH, f'//*[@id="real_contents"]/div/div[1]/div[2]/div/div[2]/div[2]/table/tbody/tr[78]/td[{i+1}]').text)

        companies_js[company]['reve'] = reve
        companies_js[company]['oper'] = oper
        companies_js[company]['depr'] = depr
        companies_js[company]['neti'] = neti
    except NoSuchElementException:
        pass
with open('companies_info.json', 'w') as f:
    json.dump(companies_js, f, ensure_ascii=False)


100%|██████████| 35317/35317 [23:53:20<00:00,  2.44s/it]   
